In [5]:
import pandas as pd

In [9]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
spark.sparkContext

<SparkContext master=local[*] appName=pyspark-shell>

In [21]:
col_names = ["id", "time", "latitude", "longitude", "direction", "road", "traffic_status",
    "avg_vehicle_speed", "vehicle_flow_rate", "traffic_concentration"]

df = spark.read.option("delimiter", ";").csv("datexDataB40.csv")
df = df.toDF(*col_names).show()
df


+---------------+--------------------+---------+---------+------------------+----+--------------+-----------------+-----------------+---------------------+
|             id|                time| latitude|longitude|         direction|road|traffic_status|avg_vehicle_speed|vehicle_flow_rate|traffic_concentration|
+---------------+--------------------+---------+---------+------------------+----+--------------+-----------------+-----------------+---------------------+
|   B40.HR.18089|2019-11-19T07:39:...|49.493904|5.9472966|  outboundFromTown| B40|     congested|             58.0|              114|                  0.0|
|   B40.HR.18260|2019-11-19T07:39:...|49.493275|5.9494343|  outboundFromTown| B40|     congested|             61.0|              108|                  1.0|
|   B40.HR.18610|2019-11-19T07:39:...| 49.49162| 5.953512|  outboundFromTown| B40|     congested|             69.0|              133|                  0.0|
|   B40.RH.18260|2019-11-19T07:39:...|49.493332|5.9494743|inboun